In [1]:
import os
import json
import requests

from azureml.core import Workspace
from azureml.core.model import Model
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice

In [2]:
subscription_id = '7a246175-8f45-456f-8c2b-3b97f145a427'
resource_group = 'rg-dps-ai'  
workspace_name = 'aml_ws_ml_forecasting'
region = 'northeurope'

In [3]:
ws = Workspace.create(name=workspace_name,
                      subscription_id=subscription_id,
                      resource_group=resource_group,
                      location=region)

print(f'Workspace {workspace_name} created')

Deploying StorageAccount with name amlwsmlfstorage7c6ae7390.
Deploying KeyVault with name amlwsmlfkeyvaulte45c3988.
Deploying AppInsights with name amlwsmlfinsights80dbecfb.
Deployed AppInsights with name amlwsmlfinsights80dbecfb. Took 2.52 seconds.
Deployed KeyVault with name amlwsmlfkeyvaulte45c3988. Took 18.8 seconds.
Deploying Workspace with name aml_ws_ml_forecasting.
Deployed StorageAccount with name amlwsmlfstorage7c6ae7390. Took 23.0 seconds.
Deployed Workspace with name aml_ws_ml_forecasting. Took 38.47 seconds.
Workspace aml_ws_ml_forecasting created


In [4]:
# ws = Workspace(subscription_id,
#                resource_group,
#                workspace_name
#               )

In [4]:
model_file = 'forecast.pkl'
model_name = 'accident_forecasting_model'

In [5]:
registered_model = Model.register(model_path=model_file, model_name=model_name, workspace=ws)

Registering model accident_forecasting_model


In [6]:
# env = Environment('forecast-env')
# conda_packages = ['numpy']
# pip_packages = ['statsmodels', 'joblib', 'json', 'azureml-sdk', 'azureml-defaults']
# env.python.conda_dependencies = CondaDependencies.create(conda_packages=conda_packages, pip_packages=pip_packages, python_version='3.9.19')

In [8]:
# env = Environment.from_existing_conda_environment('test', 'dps_env')

Exporting conda specifications for existing conda environment: dps_env


In [6]:
# Create a Conda environment for your scikit-learn model
conda_env = Environment('my-conda-env')
# conda_env.python.conda_dependencies = CondaDependencies.create(conda_packages=['scikit-learn'], pip_packages=['statsmodels', 'azureml-defaults'])
conda_env.python.conda_dependencies = CondaDependencies.create(conda_packages=['scikit-learn'])

In [7]:
inference_config = InferenceConfig(entry_script='score.py', environment=conda_env)

In [8]:
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

In [9]:
service = Model.deploy(workspace=ws,
                       name='forecasting-service',
                       models=[registered_model],
                       inference_config=inference_config,
                       deployment_config=aci_config)
service.wait_for_deployment(show_output=True)

/var/folders/j0/n9qvdq0943xfl84fpbb8p5fr0000gn/T/ipykernel_5604/4244907278.py:1: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(workspace=ws,


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2024-06-27 17:35:46+02:00 Creating Container Registry if not exists..
2024-06-27 17:45:46+02:00 Registering the environment.
2024-06-27 17:45:48+02:00 Building image..
2024-06-27 17:54:02+02:00 Generating deployment configuration.
2024-06-27 17:54:03+02:00 Submitting deployment to compute..
2024-06-27 17:54:08+02:00 Checking the status of deployment forecasting-service..
2024-06-27 17:55:32+02:00 Checking the status of inference endpoint forecasting-service.
Failed


Service deployment polling reached non-successful terminal state, current service state: Unhealthy
Operation ID: f6f07bab-50cc-4ef9-812a-94deafc8f889
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Error in entry script, ModuleNotFoundError: No module named 'statsmodels', please run print(service.get_logs()) to get details.",
  "details": [
    {
      "code": "CrashLoopBackOff",
      "message": "Error in entry script, ModuleNotFoundError: No module named 'statsmodels', please run print(service.get_logs()) to get details."
    }
  ]
}



WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Unhealthy
Operation ID: f6f07bab-50cc-4ef9-812a-94deafc8f889
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Error in entry script, ModuleNotFoundError: No module named 'statsmodels', please run print(service.get_logs()) to get details.",
  "details": [
    {
      "code": "CrashLoopBackOff",
      "message": "Error in entry script, ModuleNotFoundError: No module named 'statsmodels', please run print(service.get_logs()) to get details."
    }
  ]
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Service deployment polling reached non-successful terminal state, current service state: Unhealthy\nOperation ID: f6f07bab-50cc-4ef9-812a-94deafc8f889\nMore information can be found using '.get_logs()'\nError:\n{\n 

In [13]:
print(service.get_logs())

None


In [14]:
service

AciWebservice(workspace=Workspace.create(name='aml_ws_forecast', subscription_id='7a246175-8f45-456f-8c2b-3b97f145a427', resource_group='rg-dps-ai'), name=forecasting-service, image_id=None, image_digest=None, compute_type=ACI, state=Unhealthy, scoring_uri=http://810dce4e-2ff2-4967-8d72-ef0d814d4aa8.northeurope.azurecontainer.io/score, tags={}, properties={'azureml.git.repository_uri': 'git@github.com:induilanchezian/AI_challenge_24.git', 'mlflow.source.git.repoURL': 'git@github.com:induilanchezian/AI_challenge_24.git', 'azureml.git.branch': 'main', 'mlflow.source.git.branch': 'main', 'azureml.git.commit': '3e6a5c95a2be1b08b106cb2f653d75506a09ad7d', 'mlflow.source.git.commit': '3e6a5c95a2be1b08b106cb2f653d75506a09ad7d', 'azureml.git.dirty': 'True', 'hasInferenceSchema': 'False', 'hasHttps': 'False', 'authEnabled': 'False'}, created_by={'userObjectId': 'f63b62b2-69d5-4af4-af78-f9648abd8f8d', 'userPuId': '100320039B136511', 'userIdp': 'live.com', 'userAltSecId': '1:live.com:0003BFFD7F2C0

In [15]:
scoring_uri = service.scoring_uri
scoring_uri

'http://810dce4e-2ff2-4967-8d72-ef0d814d4aa8.northeurope.azurecontainer.io/score'

In [17]:
import requests
import json

# # Define the input data as a list
# input_data = [5,166,72,19,175,25.8,0.587,51]


# # Convert the input data to a JSON string
# input_data_json = json.dumps({"data": [input_data]})

input_data_json = json.dumps({"year": "2021", "month": "10"})

# Replace with the URL of your deployed service
scoring_uri = 'http://810dce4e-2ff2-4967-8d72-ef0d814d4aa8.northeurope.azurecontainer.io/score'  

# Define the content type
headers = {"Content-Type": "application/json"}

# Send an HTTP POST request to the endpoint
response = requests.post(scoring_uri, data=input_data_json, headers=headers)

# Check the response status code
if response.status_code == 200:
    # Parse the JSON response
    result = json.loads(response.json())
    print(result)
    # Extract the prediction (result) from the response
    prediction = result["Prediction"][0]
    print(f"Prediction: {prediction}")
else:
    print(f"Error: {response.text}")

Error: Post "http://localhost:5001/score": dial tcp 127.0.0.1:5001: connect: connection refused


In [25]:
from azureml.core.model import Model

## Deleting

In [10]:
# Delete the service

service_name = "forecasting-service"

service = ws.webservices[service_name]
service.delete()

Running
2024-06-27 19:08:13+02:00 Check and wait for operation (f6f07bab-50cc-4ef9-812a-94deafc8f889) to finish.
2024-06-27 19:08:16+02:00 Deleting service entity.
Succeeded


In [11]:
# Specify the name of the registered model to delete
model_name = 'accident_forecasting_model'

# Delete the model
registered_model = ws.models[model_name]
registered_model.delete()

In [ ]:
# Delete the workspace
ws.delete(delete_dependent_resources=True)